In [ ]:
from tqdm.notebook import tqdm
import umap
import wandb

In [ ]:
import utils
from experiments import *

In [ ]:
wandb.init(project="trajectory-visualization")
gifs_count = 10
for exp in tqdm(common_experiments):
    gifs = exp.animate_trajectories(gifs_count)
    for i, gif in enumerate(gifs):
        fname = f'gifs/{exp.experiment_name}{i}.gif'
        gif[0].save(fname,
                    save_all=True, append_images=gif[1:], optimize=False, duration=30, loop=0)
        wandb.log({f"{exp.experiment_name} animation": wandb.Video(fname, caption=exp.experiment_name)})
wandb.finish()

In [ ]:
wandb.init(project="trajectories")
for e in common_experiments:
    traj = e.single_trajectory(42)
    wandb.log({f"{e.experiment_name} data": wandb.Table(columns=e.column_names, data=traj)})
wandb.finish()

In [ ]:
wandb.init(project="kepler 3d embedding")
model = umap.UMAP(n_neighbors=250, min_dist=0.0, n_components=3)
new_traj = model.fit_transform(KeplerProblem.single_trajectory(42))
wandb.log({"kepler 3d embedding": wandb.Object3D(new_traj)})
wandb.finish()